# Query and load dobjects

In [ ]:
import lamindb as ln

db.nb.header()

Assume we're a different user who wants to continue the work of `test-user1` on the iris dataset.

In [ ]:
!lndb login test-user2

For simple frequent queries, LaminDB has a high-level query API around SQLModel.[^sqlmodel]

[^sqlmodel]: We'll also show how to use SQLModel for arbitrarily complicated queries.

In [ ]:
dobjects = ln.db.query.dobject(name="iris")

The high-level query API always returns a list of results:

In [ ]:
dobjects

There is only a single query result:

In [ ]:
dobject = dobjects[0]

Let's load a dobject into memory to analyze it:

In [ ]:
df = ln.db.load(dobject)

In [ ]:
import seaborn as sns

sns.regplot(x=df.columns[0], y=df.columns[2], data=df);

During your analysis, you notice something strange, and you'd like to understand where the data came from.

Any dobject has a data source in form of a `dtransform_id`.

In [ ]:
dobject.dtransform_id

Let's query for it via a SQL session:

In [ ]:
with db.session() as session:
    dtransform = session.get(db.schema.core.dtransform, dobject.dtransform_id)

Inspecting the result we see that the dobject originates from a Jupyter Notebook.

In [ ]:
dtransform

In [ ]:
with db.session() as session:
    jupynb = session.get(
        db.schema.core.jupynb,
        (dtransform.jupynb_id, dtransform.jupynb_v),  # it's version "1" see jupynb_v
    )

In [ ]:
jupynb

Now we found the person who last edited the notebook!

In [ ]:
with db.session() as session:
    user = session.get(db.schema.core.user, jupynb.user_id)

In [ ]:
user

Note that you can construct arbitrarily complicated SQL queries, and that we just saw queries for the primary key of an entity in the core schema module. More on this later.